In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install cornac

In [ ]:
!pip install recommenders

In [ ]:
import os
import sys
import cornac
import pandas as pd

from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.utils.notebook_utils import store_metadata

In [ ]:

df = pd.read_csv('df_movie_sen_score.csv')

In [ ]:
df.shape

In [ ]:
data = df[['userID', 'itemID', 'rating']].to_records(index=False)
data_tuples = list(data)

In [ ]:
from cornac.eval_methods import RatioSplit
from cornac.models import BPR
from cornac.metrics import Precision, Recall, NDCG
from cornac.experiment import Experiment

rs = RatioSplit(data=data_tuples, test_size=0.1, val_size=0.1, rating_threshold=4.0, seed=123)

In [ ]:

prc10 = cornac.metrics.Precision(10)
rec10 = cornac.metrics.Recall(10)
ndcg10 = cornac.metrics.NDCG(10)
mse = cornac.metrics.MSE()
rmse = cornac.metrics.RMSE()
map = cornac.metrics.MAP()

metrics=[rec10,prc10,ndcg10,mse,rmse,map]
bpr = BPR(k=50, max_iter=2000, learning_rate=0.1, lambda_reg=0.01, seed=123)

In [ ]:
bpr.fit(rs.train_set)


experiment = Experiment(eval_method=rs, models=[bpr], metrics=metrics)
experiment.run()

In [ ]:
# Retrieve user embeddings
user_embeddings = bpr.u_factors

# Retrieve item embeddings
item_embeddings = bpr.i_factors

print("User Embeddings:")
print(user_embeddings)

print("Item Embeddings:")
print(item_embeddings)

In [ ]:

user_df = pd.DataFrame(user_embeddings)
user_df.index = rs.train_set.user_ids
user_df.columns = [f"factor_{i}" for i in range(user_df.shape[1])]

# Add the user IDs as the first column
user_df.insert(0, "userID", user_df.index)

print("User Embeddings DataFrame:")
print(user_df.head())

In [ ]:
user_df

In [ ]:

item_df = pd.DataFrame(item_embeddings)
item_df.index = rs.train_set.item_ids
item_df.columns = [f"factor_{i}" for i in range(item_df.shape[1])]

# Add the item IDs as the first column
item_df.insert(0, "itemID", item_df.index)

print("Item Embeddings DataFrame:")
print(item_df.head())



In [ ]:
user_df

In [ ]:
item_df

In [ ]:

# Export user embeddings
user_embeddings_df = pd.DataFrame(user_df)
user_embeddings_df.to_csv('user_embeddings_movie_sen50.csv', index=False)

# Export item embeddings
item_embeddings_df = pd.DataFrame(item_df)
item_embeddings_df.to_csv('item_embeddings_movie_sen50.csv', index=False)